## 파이썬 완정정복 스페셜 메소드


In [1]:
import sys

In [2]:
sys.version_info

sys.version_info(major=3, minor=6, micro=6, releaselevel='final', serial=0)

In [3]:
class MyClass :
    def __init__(self, name) :
        self.name = name

In [4]:
x = MyClass("dahl")

In [5]:
str(x)

'<__main__.MyClass object at 0x00000000063D1278>'

In [6]:
repr(x)

'<__main__.MyClass object at 0x00000000063D1278>'

In [7]:
class MyClass_ :
    def __init__(self, name) :
        self.name = name
        
    def __str__(self) :
        return self.__class__.__name__ + self.name
    
    def __repr__(self) :
        s = self.__class__.__name__ + self.name
        return " {} ".format(s)

In [8]:
x = MyClass_("dahl")

In [9]:
str(x)

'MyClass_dahl'

In [10]:
repr(x)

' MyClass_dahl '

In [11]:
help(object.__str__)
# str일 경우 변수로 인식

try : 
    v = eval(str('st'))
    
except Exception as e :
    print(e)

Help on wrapper_descriptor:

__str__(self, /)
    Return str(self).

name 'st' is not defined


In [12]:
help(object.__repr__)
# repr 사용시 문자열값으로 인식
v = eval(repr('st'))
print(v, type(v))

Help on wrapper_descriptor:

__repr__(self, /)
    Return repr(self).

st <class 'str'>


### 수학산식 처리 

#### 좌측 연산과 우측연산에 대해 정의

In [13]:
class RInt:
    def __init__(self, value):
        self.value = value
        
    def __add__(self, other):
        print("__add__")
        return self.value + other
    
    def __radd__(self, other):
        print("__radd__")
        return other + self.value

In [14]:
i = RInt(100)
print(i + 88)


__add__
188


In [15]:
i = i.__radd__(99)
print(i)

__radd__
199


#### 정수와 인스턴스에 덧셈 연산을 할 때 할당된 인스턴스를 기준으로 우측연산처리 

In [16]:
i = RInt(100)

i = 99 + i
print(i)

__radd__
199


### 새로은 클래스를 정의할 때 operator 모듈을 이용해서 정의 

In [17]:
import operator as op


class Int:
    def __init__(self, value):
        self.value = value
        
    def __iadd__(self, other):
        print("__iadd__")
        return self.value + other
    
    def __truediv__(self, other):
        print("__truediv__")
        return op.truediv(self.value, other)
    
    def __floordiv__(self, other):
        print("__floordiv__")
        return op.floordiv(self.value, other)
    
    def __itruediv__(self, other):
        print("__itruediv__")
        return op.itruediv(self.value, other)
    
    def __ifloordiv__(self, other):
        print("__ifloordiv__")
        return op.ifloordiv(self.value, other)

#### 인스턴스를 만든다 

In [18]:
i = Int(100)

In [19]:
i += 99
print(i)

__iadd__
199


In [20]:
i = Int(100)
print(type(i))
print(i/3)

i /= 3
print(i)

<class '__main__.Int'>
__truediv__
33.333333333333336
__itruediv__
33.333333333333336


In [21]:
i = Int(100)
print(type(i))
print(i//3)
i //= 3
print(i)

<class '__main__.Int'>
__floordiv__
33
__ifloordiv__
33


### 클래스 생성에 대한 연산자 

In [22]:
class A:
    def __new__(cls):
        self = object.__new__(cls)
        return self
    
    def __init__(self, name):
        self.name = name
        
    def __del__(self):
        del self
        

In [23]:
a = A.__new__(A)
print(a)
A.__init__(a, 'dahl')
print(a.name)

dahl


In [24]:
del a

### 생성자와 소멸자 미정할 때 처리 방식 

In [25]:
class A:
    def __init__(self, name):
        self.name = name
        
    def amethod(self):
        print("aaaaa")
        

#### 생성자는 없으면 상위 클래스에서 가져옴

In [26]:
print(A)
ai = A.__new__(A)
print(ai.__dict__)


<class '__main__.A'>
{}


#### 초기화는 사용자 클래스에 정의가 되어야 함 

In [27]:
ai.__init__('moon')
print(ai.__dict__)

{'name': 'moon'}


In [28]:
print(ai)
ai.amethod()

aaaaa


#### 소멸자도 상위클래스에서 가져와 처리함

In [29]:
del ai

### 객체 내부 접근해 보기 


In [30]:
class Old:
    old_a = 'oldstyle'

In [31]:
o = Old()

#### 점연산자 접근하기

In [32]:
print(o.old_a)

oldstyle


In [33]:
print(o.__getattribute__('old_a'))

oldstyle


#### 변경해서 조회하기 

In [34]:
print(o.__setattr__("age", 50), o.__getattribute__('age'))
print(o.__dict__)

None 50
{'age': 50}


#### 삭제해서 확인하기

In [35]:
print(o.__delattr__('age'))
print(o.__dict__)

None
{}


### 검색연산자 확인하기 

In [36]:
class Indexing:
    def __init__(self, content):
        self.content = content
        
    def __getitem__(self, key):
        print("__getitem__ call")
        result = ""
        for i in range(len(self.content)):
            if i == key:
                result = self.content[key]
                break
        if len(self.content) < key:
            raise IndexError(key)
        return result

In [37]:
s = "Indexing search"

i = Indexing(s)
print(i[0])

__getitem__ call
I


#### 인덱스 범위가 넘어가면 예외 발생 

In [38]:
try :
    print(i[30])
except Exception as e :
    print(e)

__getitem__ call
30


### 인덱싱 검색에 예외 처리 잡기 

#### 인덱싱의 예외를 처리하기 위해 __missing__ 정의

In [39]:
class Indexing1:
    def __init__(self, content):
        self.content = content
        
    def __getitem__(self, key):
        print("__getitem__ call")
        try:
            result = self.content[key]
        except IndexError:
            result = self.__missing__(key)
        return result
    
    def __missing__(self, key):
        print("__missing__")
        return "no key" + (key if type(key) == str else str(key))

In [40]:
s = "Indexing search"

i = Indexing1(s)
print(i[0])

__getitem__ call
I


In [41]:
print(i[30])

__getitem__ call
__missing__
no key30


### 갱신과 삭제를 위한 인덱싱 처리 

#### 클래스 정의할 때 갱신과 삭제 메소드 정의

In [42]:
class Indexing1:
    def __init__(self, content):
        self.content = content
        
    def __getitem__(self, key):
        return self.content[key]
    
    def __setitem__(self, key, value):
        a = [x for x in self.content]
        if a[key] == value:
            pass
        else:
            a.insert(key, value)
        self.content = "".join(a)
    
    def __delitem__(self, key):
        a = [x for x in self.content if self[key] != x]
        self.content = "".join(a)

In [43]:
i = Indexing1("갱신 및 삭제")

del i[0]
print(i.content)

신 및 삭제


### 포함 관계 및 반복자 처리

In [44]:
l = [1, 2, 4]
m = {'k': 1, 'm': 3, 'l': 2}

print(l, l.__contains__(2))
print(m, m.__contains__('1'))


[1, 2, 4] True
{'k': 1, 'm': 3, 'l': 2} False


In [45]:
2 in l

True

#### 딕셔너리는 키만 포함관계 처리

In [46]:
'1' in m

False

In [47]:
'k' in m

True

#### 반복자 처리

In [48]:
lr = l.__reversed__()
print(lr, l)
for i in lr.__iter__():
    print(i)
    
print(l.__len__(), m.__len__())

<list_reverseiterator object at 0x0000000006405B70> [1, 2, 4]
4
2
1
3 3


#### 반복자 함수로 처리하기

In [49]:
reversed(l)

In [50]:
for i in reversed(l) :
    print(i)

4
2
1


In [51]:
iter(l)

In [52]:
for i in iter(l) :
    print(i)

1
2
4


### len 처리하기

In [53]:
class Len:
    def __init__(self, content):
        self.content = content
        
    def __len__(self):
        return len(self.content)


In [54]:
#### len 함수 처리

In [55]:
l = Len("원소의 갯수 확인")

print(len(l))

9


In [56]:
l.__len__()

9

### 반복자 클래스 만들기

In [57]:
class Iter:
    count = 0
    def __init__(self, d):
        self.d = d
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if Iter.count == len(self.d):
            raise StopIteration
        else:
            j = Iter.count
            Iter.count += 1
            return self.d[j]
        

#### 예외없이 처리하기

In [58]:
a = Iter([1, 2, 3])
b = iter(a)
print(next(b))
print(next(b))
print(next(b))
print(next(b, None))

1
2
3
None


In [59]:
try :
    next(b) 
except Exception as e :
    print(e)

### 객체 접근연산 처리 

In [60]:
class Dot:
    def __init__(self):
        self.test = 1
        self.test2 = 2
        
    def __getattribute__(self, attr):
        print("__getattribute__", attr)
        return object.__getattribute__(self, attr)

In [61]:
d = Dot()
print(d.test)
print(d.test2)

__getattribute__ test
1
__getattribute__ test2
2


#### 속성이 없을 경우 처리 

In [62]:
try :
    print(d.test3)
    
except Exception as e :
    print(e)

__getattribute__ test3
'Dot' object has no attribute 'test3'


#### object 점연산 메소드 

In [63]:
help(object.__getattribute__)

Help on wrapper_descriptor:

__getattribute__(self, name, /)
    Return getattr(self, name).



### 점연산자 예외 처리 추가 

In [64]:
class Dot:
    def __init__(self):
        self.test = 1
        self.test2 = 2
        
    def __getattribute__(self, attr):
        print("__getattribute__", attr)
        return object.__getattribute__(self, attr)
    
    def __getattr__(self, attr):
        print("__getattr__", attr)
        try:
            self.__dict__[attr]
        except KeyError as e:
            return 'Key Error ' + e.args[0]

In [65]:
d = Dot()
print(d.test)
print(d.test2)

__getattribute__ test
1
__getattribute__ test2
2


#### 없는 속성을 조회

In [66]:
print(d.test3)

__getattribute__ test3
__getattr__ test3
__getattribute__ __dict__
Key Error test3


### 점연산자 갱신하기

In [67]:
help(object.__setattr__)

Help on wrapper_descriptor:

__setattr__(self, name, value, /)
    Implement setattr(self, name, value).



#### 점연산자 없는 클래스 정의 

In [68]:
class Old:
    old_a = "oldstyle"
    

#### 상위크래스의 점연산자를 가지고 처리

In [69]:
o = Old()
print(Old.__dict__)
print(o.old_a)
print(o.__getattribute__('old_a'))
print(o.__setattr__('age', 50), o.__getattribute__('age'))
print(o.__dict__)
print(o.__delattr__('age'))
print(o.__dict__)

{'__module__': '__main__', 'old_a': 'oldstyle', '__dict__': <attribute '__dict__' of 'Old' objects>, '__weakref__': <attribute '__weakref__' of 'Old' objects>, '__doc__': None}
oldstyle
oldstyle
None 50
{'age': 50}
None
{}


In [70]:
help(object.__delattr__)

Help on wrapper_descriptor:

__delattr__(self, name, /)
    Implement delattr(self, name).



#### getattr 내장함수를 통해 점연산자 호출

In [71]:
try :
    getattr(o, "major")
except Exception as e :
    print(e)

'Old' object has no attribute 'major'


In [72]:
print(getattr(o, 'major', "CS"))

CS


### 사용자 클래스에 점 연산자를 접근해서 갱신하기

In [73]:
class Empty:
    def __setattr__(self, key, value):
        print("__setattr__")
        self.__dict__[key] = value

In [74]:
d = [
    {'name': '홍길동', "age": 33},
    {'name': '문길동', "age": 33},
    {'name': '김길동', "age": 33}    
]

e_class = []

for elem in d:
    e = Empty()
    for k, v in elem.items():
        setattr(e, k, v)
    e_class.append(e)

__setattr__
__setattr__
__setattr__
__setattr__
__setattr__
__setattr__


### 호출연산 확인하기

In [75]:
class B:
    def c(self):
        pass


def a():
    return "function a"



In [76]:
print(callable(B))
print(callable(a))

print(callable(B.c))

True
True
True


#### 인스턴스 호출 확인하기 

In [77]:
class A:
    def c(self):
        pass
    
    
class B:
    def __call__(self):
        pass
    
    def c(self):
        pass
    

In [78]:
b = B()
print(callable(b))
a = A()
print(callable(a))

True
False


### 컨텍스트 환경에 맞는 처리

#### 파일을 처리할 때 실제 클로즈없이 처리 가능

In [79]:
with open("foo1.txt", 'w') as f:
    f.write('Life is too short, you need python')
    
    
with open("foo1.txt", 'r') as f1:
    for i in f1:
        print(i)

Life is too short, you need python


#### 일반 파일을 읽고 컨텍스트 처리

In [80]:
file = open('foo1.txt', 'r')

In [81]:
help(file.__enter__)
help(file.__exit__)

Help on built-in function __enter__:

__enter__(...) method of _io.TextIOWrapper instance

Help on built-in function __exit__:

__exit__(...) method of _io.TextIOWrapper instance



In [82]:
file.__exit__(None, None, None)


In [83]:
try :
    print(file.read(1))
except Exception as e :
    print(e)

I/O operation on closed file.


### Sequence 클래스를 만들어서 확인하기

In [84]:
class Seq:
    def __init__(self, *args):
        self.com = list(args)
        
    def __len__(self):
        return len(self.com)
    
    def __getitem__(self, index):
        return index
    

In [85]:
s = Seq(1, 2, 3, 4)
print(s[0])
print(s[1:2])
print(s[0:3:1])

0
slice(1, 2, None)
slice(0, 3, 1)


### Sequence 정보 가져오기 

In [86]:
class Seq:
    def __init__(self, *args):
        self.com = list(args)
        
    def __len__(self):
        return len(self.com)
    
    def __getitem__(self, index):
        return self.com[index]
    


In [87]:
s = Seq(1, 2, 3, 4)
print(s[0])
print(s[1:2])
print(s[0:3:1])

1
[2]
[1, 2, 3]
